<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/Acme.jpeg width=100 align="left">
<img src=images/SingaporeMBS2.png>

## Vaults Setup

**Cleanup Old Files**

In [ ]:
alias vault='/usr/local/bin/vaulte'

In [ ]:
#rm $HOME/demos/vault_cluster/vrd/vault_init_output
rm -rf $HOME/demos/vault_cluster/vrd/data/*
#rm -r $HOME/demos/vault_cluster/vrd/vrd*

In [ ]:
#rm $HOME/demos/vault_cluster/vrd2/vault_init_output
rm -r $HOME/demos/vault_cluster/vrd2/data/*
#rm -r $HOME/demos/vault_cluster/vrd2/vrd*
#tree $HOME/demos/vault_cluster/vrd2

### Begin Performance Replication

In [ ]:
vault version

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
export VAULT_ADDR2=http://127.0.0.1:8202
export VAULT_SKIP_VERIFY=true

In [ ]:
env |grep -i vault

### Start Two Vault Servers

**Populate Vault Server configuration files**

In [ ]:
mkdir -p $HOME/demo/vault_cluster/vrd/data/raft
cat <<EOF > $HOME/demos/vault_cluster/vrd/vrd_config.hcl
storage "raft" {
  path = "$HOME/demos/vault_cluster/vrd/data"
  node_id = "demo"
}
listener "tcp" {
  address = "0.0.0.0:8200"
  cluster_address = "0.0.0.0:8201"
  tls_disable = "true"
}
api_addr = "https://127.0.0.1:8200"
cluster_addr = "https://127.0.0.1:8201"
disable_mlock="false"
disable_cache="false"
ui = "true"
max_lease_ttl="24h"
default_lease_ttl="1h"
raw_storage_endpoint=true
cluster_name="hashi-vault"
insecure_tls="true"
plugin_directory="$HOME/demos/vault_cluster/vrd/data/plugins"
EOF

In [ ]:
mkdir -p $HOME/demos/vault_cluster/vrd2/
cat <<EOF > $HOME/demos/vault_cluster/vrd2/vrd2_config.hcl
storage "raft" {
  path = "$HOME/demos/vault_cluster/vrd2/data"
  node_id = "demo"
}
listener "tcp" {
  address = "0.0.0.0:8202"
  cluster_address = "0.0.0.0:8203"
  tls_disable = "true"
}
api_addr = "https://127.0.0.1:8202"
cluster_addr = "https://127.0.0.1:8203"
disable_mlock="false"
disable_cache="false"
ui = "true"
max_lease_ttl="24h"
default_lease_ttl="1h"
raw_storage_endpoint=true
cluster_name="hashi-vault"
insecure_tls="true"
plugin_directory="$HOME/demos/vault_cluster/vr2/data/vault/plugins"
EOF

**AUTO RUN ABOVE**

**EXECUTE EACH COMMAND IN SEPARATE TERMINAL WINDOW**

`vaulte server -config=$HOME/demos/vault_cluster/vrd/vrd_config.hcl`

`vaulte server -config=$HOME/demos/vault_cluster/vrd2/vrd2_config.hcl`

### Initialize Vaults
(save output to file for demo use) 

Note: the initialization takes few seconds to complete

In [ ]:
curl --request POST --data '{"secret_shares": 1, "secret_threshold": 1}' $VAULT_ADDR/v1/sys/init > ~/demos/vault_cluster/vrd/vault_init_output

In [ ]:
curl --request POST --data '{"secret_shares": 1, "secret_threshold": 1}' $VAULT_ADDR2/v1/sys/init > ~/demos/vault_cluster/vrd2/vault_init_output

**Parse output to variables** (for demo use)

In [ ]:
export VAULT_ROOT_TOKEN=$(cat ~/demos/vault_cluster/vrd/vault_init_output | jq -r '.root_token')
export VAULT_KEY_BASE64=$(cat ~/demos/vault_cluster/vrd/vault_init_output | jq -r '.keys_base64[0]' )
echo $VAULT_ROOT_TOKEN
echo $VAULT_KEY_BASE64

In [ ]:
export VAULT_ROOT_TOKEN2=$(cat ~/demos/vault_cluster/vrd2/vault_init_output | jq -r '.root_token')
export VAULT_KEY_BASE642=$(cat ~/demos/vault_cluster/vrd2/vault_init_output | jq -r '.keys_base64[0]' )
echo $VAULT_ROOT_TOKEN2
echo $VAULT_KEY_BASE642

### Unseal Vaults

In [ ]:
curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_ROOT_TOKEN" \
    --request PUT \
    --data '{ "key": "'$VAULT_KEY_BASE64'" }' \
    $VAULT_ADDR/v1/sys/unseal

In [ ]:
  curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_ROOT_TOKEN2" \
    --request PUT \
    --data '{ "key": "'$VAULT_KEY_BASE642'" }' \
    $VAULT_ADDR2/v1/sys/unseal

### License Vaults

In [ ]:
curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_ROOT_TOKEN" \
    --request PUT \
    --data @/Users/tio/Documents/vault_license.json \
    $VAULT_ADDR/sys/license

In [ ]:
curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_ROOT_TOKEN2" \
    --request PUT \
    --data "@/Users/tio/Documents/vault_license.json" \
    $VAULT_ADDR2/sys/license

# Interactive Demo: Enable Performance Replication

**Copy Token to Pasteboard for UI Login**

In [ ]:
echo $VAULT_ROOT_TOKEN | pbcopy
echo $VAULT_ROOT_TOKEN

In [ ]:
echo $VAULT_ROOT_TOKEN2 | pbcopy
echo $VAULT_ROOT_TOKEN2

**Generate NEW root key for Performance Replication cluster**

In [ ]:
## Generate One Time Password (OTP) Needed to Generate DR token
#alias vault_secondary='VAULT_UI=true VAULT_ADDR=http://127.0.0.1:8202 vault'
#VAULT_SECONDARY_ADDR='127.0.0.1:8202'
#ONE_TIME_PASSWORD=$(vault_secondary operator generate-root -generate-otp)
ONE_TIME_PASSWORD=$(vault operator generate-root -generate-otp VAULT_ADDR=http://127.0.0.1:8202)
echo $ONE_TIME_PASSWORD

In [ ]:
curl --header "X-Vault-Token: $VAULT_ROOT_TOKEN" --request PUT --data '{"otp":"'"${ONE_TIME_PASSWORD}"'"}' 127.0.0.1:8202/v1/sys/generate-root/attempt

In [ ]:
## Start Generation of DR Operation Token Attempt -- Get NONCE to give to UNSEAL KEY holders
NONCE=$(curl --header "X-Vault-Token: $VAULT_ROOT_TOKEN" --request PUT --data '{"otp":"'"${ONE_TIME_PASSWORD}"'"}' 127.0.0.1:8202/v1/sys/generate-root/attempt | jq --raw-output '.nonce')
echo $NONCE

In [ ]:
## Get Your ENCODED TOKEN that will be combined with OTP to produce New Root Token for Perf Replica cluster
export UNSEAL_KEY=$(cat ~/demos/vault_cluster/vrd/vault_init_output | jq -r '.keys[0]')
echo $UNSEAL_KEY
ENCODED_TOKEN=$(curl -s --header "X-Vault-Token: ${VAULT_ROOT_TOKEN}" --request PUT --data '{"key":"'"${UNSEAL_KEY}"'", "nonce":"'"${NONCE}"'"}' http://127.0.0.1:8202/v1/sys/generate-root/update | jq --raw-output '.encoded_token')
echo $ENCODED_TOKEN
#PR_ROOT_TOKEN=$(vault_secondary operator generate-root -otp="${ONE_TIME_PASSWORD}" -decode="${ENCODED_TOKEN}")
PR_ROOT_TOKEN=$(VAULT_ADDR=http://127.0.0.1:8202 vault operator generate-root -otp="${ONE_TIME_PASSWORD}" -decode="${ENCODED_TOKEN}")
echo $PR_ROOT_TOKEN

In [ ]:
echo $PR_ROOT_TOKEN | pbcopy

## Interactive Demo: Log Into Replica
## Interactive Demo: Create / Delete Secrets in Primary and View in Replica

**END**

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 class="center">